In [225]:
import numpy as np
from collections import Counter
from scipy.optimize import minimize

In [282]:
class DecisionTree:
    def __init__(self, max_depth=None):
        self.max_depth=max_depth
        
    def fit(self, X, y):
        self.X = X
        self.y = y
        self.classes = np.unique(y)
        self.features = np.arange(X.shape[1])
        
        if self.max_depth == None:
            self.max_depth = len(self.features)
        
        self.borders = np.ones(X.shape[1]*100).reshape(X.shape[1], 100)
        for i in range(len(X[0])):
            sorted_ar = np.sort(np.transpose(X)[i])
            self.borders[i] = np.linspace(sorted_ar[-2]+0.00001, sorted_ar[1]-0.00001, 100)
                    
        self.root = self.step(np.arange(len(X)), 1)
            
    def step(self, Q, depth): 
        min_j = self.features[0]
        min_t = self.borders[min_j][0]
        min_G = self.G(Q, min_j, min_t)
                
        for j in self.features:
            for t in self.borders[j]:
                cur = self.G(Q, j, t)
                
                if cur < min_G:
                    min_G = cur
                    min_j = j
                    min_t = t
        
        leaf = Tree(min_j, min_t)
        self.features = np.extract(self.features!=min_j, self.features)
        
        left = self.L(Q, min_j, min_t)
        right = self.R(Q, min_j, min_t)
                
        if self.features != [] and depth < self.max_depth and len(left)!=0 and len(right)!=0:
            leaf.left = self.step(left, depth+1)
            self.features = np.append(self.features, min_j)
            leaf.right = self.step(right, depth+1)
            self.features = np.append(self.features, min_j)
        else:
            if len(left)==0 or len(right)==0:
                leaf = Counter([y[i] for i in Q]).most_common(1)[0][0]
            else:
                leaf.left = Counter([y[i] for i in left]).most_common(1)[0][0]
                leaf.right = Counter([y[i] for i in right]).most_common(1)[0][0]
        return leaf
    
    def L(self, Q, j, t): #Q - массив индексов
        return np.array([ind for ind in Q if self.X[ind][j] < t])
    
    def R(self, Q, j, t): #Q - массив индексов
        return np.array([ind for ind in Q if self.X[ind][j] >= t])
            
    def G(self, Q, j, t):
        L = self.L(Q,j,t)
        R = self.R(Q,j,t)
        
        if len(L) == 0 or len(R)==0:
            return 2
        else:
            res = float(len(L))/float(len(Q)) * self.H(L) + float(len(R))/len(Q) * self.H(R)
            return res
    
    def H(self, R): # R - массив индексов из X
        cnt = Counter([self.y[i] for i in R])
        res = 0
        for k in self.classes:
            p_k = float(cnt[k]) / float(len(R))
            res += p_k * (1 - p_k)
        
        return res
            
    def print_tree(self):
        print 'Tree:'
        self.root.print_tree()
        
    def predict(self, X):
        res = np.array([])
        for line in X:
            res = np.append(res, self.root.predict(line))
        return res

In [283]:
class Tree(object):
    def __init__(self, j, t):
        self.left = None
        self.right = None
        self.j = j
        self.t = t
    
    def print_tree(self):
        print 'j:', self.j, 't:', self.t
        print 'left'
        if type(self.left)==Tree:
            self.left.print_tree()
        else:
            print 'class', self.left
        print 'right'
        if type(self.right)==Tree:
            self.right.print_tree()
        else:
            print 'class', self.right
    
    def predict(self, line):
        if line[self.j] < self.t:
            if type(self.left) == Tree:
                return self.left.predict(line)
            else:
                return self.left
        else:
            if type(self.right) == Tree:
                return self.right.predict(line)
            else:
                return self.right

In [286]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

decision_tree = DecisionTree()
X, y = make_classification(n_features = 10, n_informative=10, n_redundant=0, n_repeated=0)

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75)

decision_tree.fit(X_train, y_train)
decision_tree.print_tree()
predict = decision_tree.predict(X_test)
print float(len(predict[predict == y_test]))/float(len(y_test))

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel/__main__.py:41: DeprecationWarning: elementwise != comparison failed; this will raise an error in the future.


Tree:
j: 4 t: 1.02443668988
left
j: 5 t: 0.0975792151978
left
j: 7 t: -0.987205116977
left
j: 6 t: -2.77431106937
left
class 0
right
j: 1 t: 2.69842095538
left
j: 2 t: 2.62646662121
left
j: 3 t: 5.16218655162
left
j: 8 t: 0.857766848409
left
j: 9 t: 0.279420777617
left
class 0
right
class 1
right
class 0
right
class 1
right
class 0
right
class 1
right
j: 1 t: 3.82066314579
left
j: 6 t: -2.91960987045
left
class 0
right
class 0
right
j: 7 t: 6.72838512272
left
class 0
right
class 0
right
j: 1 t: 0.0531357922744
left
j: 7 t: 3.26196052228
left
class 0
right
class 0
right
j: 1 t: 0.21345610519
left
class 0
right
class 1
right
j: 5 t: 1.315835475
left
j: 1 t: 0.694417043937
left
class 1
right
class 1
right
class 1
0.6
